## ***CatBoost Regressor***

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("C:\\Users\\default.DESKTOP-GAN0M7C\\my_projects\\data\\amazon data\\Amazon Sale Report.csv")

C:\Users\default.DESKTOP-GAN0M7C\AppData\Local\Temp\ipykernel_9844\179680575.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:\\Users\\default.DESKTOP-GAN0M7C\\my_projects\\data\\amazon data\\Amazon Sale Report.csv")


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

### Data Cleaning & Transformation

In [4]:

def preprocess_orders(data: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the orders dataset:
    - Drop irrelevant or high-cardinality columns
    - Rename columns if needed
    - Extract useful features from 'Date'
    - Create derived features like day, month, weekday, weekend flag
    """
    
    # 1. Drop unwanted columns if they exist
    DROP_COLS = [
        "index",
        "Order ID",           # pure identifier
        "SKU",                # high cardinality ID
        "promotion-ids",      # sparse / leakage-prone
        "Unnamed: 22",
        "ASIN",
        "fulfilled-by",
        "ship-postal-code",
        "currency",
        "Size"
    ]
    
    data = data.drop(columns=[c for c in DROP_COLS if c in data.columns])
    
    # 2. Rename columns if necessary
    if 'Sales Channel ' in data.columns:
        data.rename(columns={'Sales Channel ':'Sales Channel'}, inplace=True)
    
    # 3. Convert 'Date' to datetime safely
    if 'Date' in data.columns:
        data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")
        
        # Extract date-related features
        data["order_day"] = data["Date"].dt.day_name()
        data["order_month"] = data["Date"].dt.month_name()
        data["order_weekday"] = data["Date"].dt.weekday
        data["is_weekend"] = data["order_weekday"].isin([5,6]).astype(int)
        
        # Drop original Date column
        data = data.drop(columns=["Date"])
    
    return data


In [5]:
data = preprocess_orders(data)
print(data.head())

                         Status Fulfilment Sales Channel ship-service-level  \
0                     Cancelled   Merchant     Amazon.in           Standard   
1  Shipped - Delivered to Buyer   Merchant     Amazon.in           Standard   
2                       Shipped     Amazon     Amazon.in          Expedited   
3                     Cancelled   Merchant     Amazon.in           Standard   
4                       Shipped     Amazon     Amazon.in          Expedited   

     Style       Category Courier Status  Qty  Amount    ship-city  \
0   SET389            Set            NaN    0  647.62       MUMBAI   
1  JNE3781          kurta        Shipped    1  406.00    BENGALURU   
2  JNE3371          kurta        Shipped    1  329.00  NAVI MUMBAI   
3    J0341  Western Dress            NaN    0  753.33   PUDUCHERRY   
4  JNE3671            Top        Shipped    1  574.00      CHENNAI   

    ship-state ship-country    B2B order_day order_month  order_weekday  \
0  MAHARASHTRA           IN  

C:\Users\default.DESKTOP-GAN0M7C\AppData\Local\Temp\ipykernel_9844\103245547.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")


### Feature Engineering

In [6]:
# seperating numerical and categorical columns
numerical_cols = data.select_dtypes(include="number").columns
print("numerical_cols:\n", numerical_cols)
print()

categorical_cols = data.select_dtypes(exclude="number").columns
print("catgeorical_cols:\n", categorical_cols)

numerical_cols:
 Index(['Qty', 'Amount', 'order_weekday', 'is_weekend'], dtype='object')

catgeorical_cols:
 Index(['Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style',
       'Category', 'Courier Status', 'ship-city', 'ship-state', 'ship-country',
       'B2B', 'order_day', 'order_month'],
      dtype='object')


In [7]:
numerical_cols = numerical_cols.drop(['Amount'])
numerical_cols

Index(['Qty', 'order_weekday', 'is_weekend'], dtype='object')

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

def build_catboost_preprocessor(num_cols, cat_cols):
    
    num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median"))
    ])

    cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent"))
        # NO encoding — CatBoost handles categoricals natively
    ])

    return ColumnTransformer([
        ("num", num_pipeline, num_cols),
        ("cat", cat_pipeline, cat_cols)
    ])


In [9]:
# for linear models -> scaling is on!
preprocessor = build_catboost_preprocessor(
    num_cols=numerical_cols,
    cat_cols=categorical_cols,
)

In [10]:
preprocessor

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [11]:

# seperating the input and target variable from data
TARGET = "Amount"   # regression target

X = data.drop(columns=[TARGET])
y = data[TARGET]


In [12]:
# fill target with median values
y.fillna(value=data['Amount'].median(), inplace=True)


In [13]:
X.shape

(128975, 16)

In [14]:
y.shape

(128975,)

In [15]:
# seperating the input and target data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


In [16]:
X_train.shape

(103180, 16)

In [17]:
y_train.shape

(103180,)

In [18]:
# apply preprocessor and take the sample records

def process_and_sample(X_train, X_test, y_train, sample_frac=0.1, n_bins=10, preprocessor=None, random_state=42):
    # 1. Fit preprocessor on full X_train
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    
    # 2. Create quantile bins for stratified sampling
    y_bins = pd.qcut(y_train, q=n_bins, duplicates="drop")
    
    # 3. Stratified sampling
    X_train_sampled, _, y_train_sampled, _ = train_test_split(
        X_train_processed,
        y_train,
        test_size=1-sample_frac,   # keep fraction for training
        stratify=y_bins,
        random_state=random_state
    )
    
    return X_train_processed, X_test_processed, X_train_sampled, y_train_sampled


In [19]:
X_train_processed, X_test_processed, X_train_sampled, y_train_sampled = process_and_sample(
    X_train, X_test, y_train, sample_frac=0.25, preprocessor=preprocessor
)

print("Full X_train_processed shape:", X_train_processed.shape)
print("Sampled X_train_sampled shape:", X_train_sampled.shape)


Full X_train_processed shape: (103180, 16)
Sampled X_train_sampled shape: (25795, 16)


### Model Training

a) training with ***entire data*** -> as the lightgbm itself very fast!

In [21]:
!pip install catboost

   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.5/102.4 MB 5.5 MB/s eta 0:00:19
    --------------------------------------- 1.6/102.4 MB 4.8 MB/s eta 0:00:22
   - -------------------------------------- 3.7/102.4 MB 6.7 MB/s eta 0:00:15
   -- ------------------------------------- 5.8/102.4 MB 7.8 MB/s eta 0:00:13
   --- ------------------------------------ 7.9/102.4 MB 8.0 MB/s eta 0:00:12
   --- ------------------------------------ 8.7/102.4 MB 7.4 MB/s eta 0:00:13
   --- ------------------------------------ 9.7/102.4 MB 7.1 MB/s eta 0:00:13
   --- ------------------------------------ 10.2/102.4 MB 6.5 MB/s eta 0:00:15
   ---- ----------------------------------- 11.0/102.4 MB 5.9 MB/s eta 0:00:16
   ---- ----------------------------------- 11.5/102.4 MB 5.7 MB/s eta 0:00:17
   ---- ----------------------------------- 12.3/102.4 MB 5.4 MB/s eta 0:00:17
   ----- ---------------------------------- 13.6/102.4 MB 5.5 MB/s 

In [22]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [28]:
# identifying categorial features indices
# categorical features come AFTER numerical features in ColumnTransformer
n_num = len(numerical_cols)

cat_feature_indices = list(range(n_num, n_num + len(categorical_cols)))


In [29]:
cat_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    loss_function="RMSE",
    eval_metric="RMSE",
    random_seed=42,
    verbose=100
)


In [30]:
cat_model.fit(
    X_train_processed,
    y_train,
    cat_features=cat_feature_indices
)


0:	learn: 264.6918805	total: 524ms	remaining: 8m 43s
100:	learn: 156.4902113	total: 31.5s	remaining: 4m 40s
200:	learn: 151.1795812	total: 1m 7s	remaining: 4m 27s
300:	learn: 148.4255011	total: 1m 47s	remaining: 4m 10s
400:	learn: 146.2846615	total: 2m 20s	remaining: 3m 29s
500:	learn: 144.7321939	total: 2m 55s	remaining: 2m 54s
600:	learn: 143.6110830	total: 3m 21s	remaining: 2m 13s
700:	learn: 142.5174263	total: 3m 49s	remaining: 1m 37s
800:	learn: 141.5253715	total: 4m 16s	remaining: 1m 3s
900:	learn: 140.4540733	total: 4m 45s	remaining: 31.3s
999:	learn: 139.7856258	total: 5m 13s	remaining: 0us


In [31]:
y_pred = cat_model.predict(X_test_processed)

metrics = {
    "MAE": mean_absolute_error(y_test, y_pred),
    "MSE": mean_squared_error(y_test, y_pred),
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "R2": r2_score(y_test, y_pred)
}

metrics


{'MAE': 85.22229898772974,
 'MSE': 20831.807006591225,
 'RMSE': np.float64(144.33227984962764),
 'R2': 0.7248776979244929}